# RNA + MS data integration

In [ ]:
library(RColorBrewer)
library(VennDiagram)

In [ ]:
# Load transformed MS data with t-test results
protFiltered = read.table("Perseus_run/ProteinsTTestFiltered.txt", sep = "\t", head=T, quote="")

In [ ]:
names(protFiltered)[2] <- "LogPvalue"
# Color for significant abundance changes
protFiltered$color = ifelse(test = protFiltered$Difference > 0, "#4A91C4", "#F09F4E")
protFiltered[protFiltered$Significant != "+", 99] = "grey"
protFiltered$Gene.names <- as.character(protFiltered$Gene.names)
# Keep only the first gene name when several are provided
protFiltered[,7] <- sub(";.*", "", protFiltered[,7])

In [ ]:
listDownMS = protFiltered[protFiltered$color == "#4A91C4",7]
listUpMS = protFiltered[protFiltered$color == "#F09F4E",7]

In [ ]:
load("RNAlists.diff")

In [ ]:
drawVennFromList = function(a, b, c, d, ...){
    draw.quad.venn(length(a), length(b), length(c), length(d),
        length(intersect(a, b)), length(intersect(a, c)), length(intersect(a, d)),
        length(intersect(b, c)), length(intersect(b, d)), length(intersect(c, d)),
        length(intersect(intersect(a, b), c)), length(intersect(intersect(a, b), d)),
        length(intersect(intersect(a, c), d)), length(intersect(intersect(c, b), d)),
        length(intersect(intersect(intersect(a, b), c), d)), ...)
}

In [ ]:
pdf("RNA_MS_overlap.pdf")
drawVennFromList(listDownMS, listDownRNA, listUpMS, listUpRNA, category = c("MS - down", "RNA-seq - down", "MS - up", "RNA-seq - up"),
                fill = brewer.pal(4, "Paired"), fontfamily = "sans", cat.fontfamily = "sans")
dev.off()

In [ ]:
for (i in intersect(listDownMS, listDownRNA)) cat(paste0(i, "\n"))

In [ ]:
for (i in union(listUpMS, listDownMS)) cat(paste0(i, "\n"))

In [ ]:
for (i in intersect(listUpMS, listUpRNA)) cat(paste0(i, "\n"))

In [ ]:
for (i in listUpRNA) cat(paste0(i, "\n"))

## Prepare PPI for OmicsIntegrator
The PPI was obtained from BioGRID Release 3.4.160 as [all the interactions available for mice in the TAB2 format](https://downloads.thebiogrid.org/Download/BioGRID/Release-Archive/BIOGRID-3.4.160/BIOGRID-ORGANISM-3.4.160.tab2.zip).

In [ ]:
PPI = read.table("OmicsIntegratorRun/BIOGRID-ORGANISM-Mus_musculus-3.4.160.tab2.txt", sep = "\t", header = T,
                quote = '', na.strings = "-", comment.char = "")
head(PPI[,c(8,9)])

In [ ]:
names(PPI)

In [ ]:
# Map an experimental system to a confidence score for direct PPI
expScore = data.frame(levels(PPI$Experimental.System), c(0.2,0.2,0.2,0.2,0.99,0.5,0.2,0.2,0.2,
  0,0,0.99,0.99,0,0.99,0,0,0,
  0.99,0.2,0.2,0.99,0,0,0,0,0.99))
expScore

In [ ]:
PPI$expScore = expScore[PPI$Experimental.System,2] # Levels = row index in the lookup table

In [ ]:
write.table(PPI[,c(8,9,25),], file = "OmicsIntegratorRun/mousePPI.tsv", col.names = F, row.names = F, sep = "\t",
            quote = F, dec = ".")

In [ ]:
sessionInfo()